# Theano Tutorial Notes

Personal notes for the official tutorial [http://deeplearning.net/software/theano/tutorial/](http://deeplearning.net/software/theano/tutorial/)

In [1]:
import theano
import theano.tensor as T
from theano import pp
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# numpy arrays
print np.array([1,2,3,4])
print np.array([1,2,3,4], copy=False)
print np.array([[1,2],[3,4]], copy=False, order='C') # A (default), F (Fortran), C (contiguous)
print np.asarray([1,2,3,4])
print np.array([1,2,3,4], dtype=complex)
print np.array([1,2,3,4], ndmin=2)
print np.array([[1,2,3],[4,5,6],[7,8,9]])
print np.matrix('1 2; 3 4') # subtype of ndarray (convenience method)

[1 2 3 4]
[1 2 3 4]
[[1 2]
 [3 4]]
[1 2 3 4]
[ 1.+0.j  2.+0.j  3.+0.j  4.+0.j]
[[1 2 3 4]]
[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1 2]
 [3 4]]


In [3]:
# theano types
x = T.fscalar('x')
print pp(x)
print pp(T.fscalar())
print type(x)
print x.type()
print T.fscalars('a','b','c','d','e')
print T.fscalar(name='foo')

x
<TensorType(float32, scalar)>
<class 'theano.tensor.var.TensorVariable'>
<TensorType(float32, scalar)>
[a, b, c, d, e]
foo


In [4]:
# function & gradient
y = 2 * x + 4
dy = theano.grad(y,x)
y_f = theano.function([x],y)
y_f = theano.function(inputs=[x], outputs=y) # same as above
dy_f = theano.function([x],dy)
print y_f(1)
print y.eval({x : 1})
print dy_f(1)

6.0
6.0
2.0


In [5]:
# matrix & vector
A = T.dmatrix('A')
x = T.dvector('x')
y = A.dot(x)
y.eval({A: np.array([[1,2],[3,4]]), x: np.array([2,0])})


array([ 2.,  6.])

In [6]:
# More functions
x = T.scalar('x')
print T.exp(x).eval({x:1})
print T.tanh(x).eval({x:1})
print T.log(x).eval({x:1})

2.71828182846
0.761594155956
0.0


In [7]:
# computing more at the same time
x = T.scalar('x')
print theano.function([x], [T.exp(x), T.tanh(x)])(1)

[array(2.718281828459045), array(0.7615941559557649)]


In [8]:
# default & named parameters
x, y, w = T.scalars('x', 'y', 'w')
f = theano.function([x,
                     theano.Param(y, default=1), 
                     theano.Param(w, default=1, name='foo')], 
                    w*(x+y))
print f(1)
print f(1, 2)
print f(1, 2, 10)
print f(1, 2, foo=10)


2.0
3.0
30.0
30.0


In [9]:
# shared variables
s = theano.shared(np.float32(0))
print s, s.eval(), s.get_value()

<TensorType(float32, scalar)> 0.0 0.0


In [10]:
# function with side effects
x = T.fscalar('x')
function_of_state = s + x
f = theano.function([x], outputs=s, updates=[(s,function_of_state)])

In [11]:
# function evaluation returns previous state. state access with #get_value()
s.set_value(0)
print s.get_value()
print f(1)
print f(1)
print s.get_value()

0.0
0.0
1.0
2.0


In [12]:
# skip shared variables
s_alt = T.fscalar()
g = theano.function([x,s_alt], function_of_state, givens=[(s,s_alt)])
print g(1,2)

3.0
